In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

In [35]:
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells.h5ad')
adata_1 = adata_T[adata_T.obs['leiden'] == '3', :]
adata_B = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_B_cells.h5ad')
adata_2 = adata_B

In [36]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_2, sample_id_col='sample_id')

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [37]:
# because some samples are missing, we need to add them to the dataset
# to make sure nosamplefiltered
from scRNA_utils import add_missing_samples
import anndata
# Extract sample_ids
sample_ids_1 = set(adata_1_pseudo.obs['sample_id'])
sample_ids_2 = set(adata_2_pseudo.obs['sample_id'])

# Identify missing sample_ids
missing_sample_ids_1 = sample_ids_2 - sample_ids_1
missing_sample_ids_2 = sample_ids_1 - sample_ids_2

# Add missing sample_ids to each dataset
adata_1_pseudo = add_missing_samples(adata_1_pseudo, missing_sample_ids_1)
adata_2_pseudo = add_missing_samples(adata_2_pseudo, missing_sample_ids_2)
# Sort the obs DataFrame by sample_id in both AnnData objects
adata_1_pseudo = adata_1_pseudo[adata_1_pseudo.obs.sort_values('sample_id').index]
adata_2_pseudo = adata_2_pseudo[adata_2_pseudo.obs.sort_values('sample_id').index]

# Verify that both datasets have the same sample_ids and in the same order
assert (adata_1_pseudo.obs['sample_id'].values == adata_2_pseudo.obs['sample_id'].values).all()

# Now adata_T_pseudo and adata_Endo_pseudo have the same sample_ids in the same order

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/merge.py:1111: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


In [38]:
adata_1_pseudo.raw = adata_1_pseudo
adata_2_pseudo.raw = adata_2_pseudo

In [39]:
DEG_1 = paird_ttest(adata_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = paird_ttest(adata_2, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [40]:
DEG_1 = DEG_1[DEG_1['pval'] < 0.05]
DEG_2 = DEG_2[DEG_2['pval'] < 0.05]
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_CD4EX' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_B' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')

In [41]:
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')
gene_df['treatment'].value_counts()

1    31
0    30
Name: treatment, dtype: int64

# group GEM

In [55]:
results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_CD4EXvsB_fisherz_nosamplefiltered.xlsx')

### add GEM to gene_df

In [56]:
# only keep the significant results with correlation > 0.3 or < -0.3
results = results[(results['g1vsg2_correlation'] > 0.3)]

In [57]:
calculate_g1_L_correlation(results, adata_1_pseudo)
calculate_g2_R_correlation(results, adata_2_pseudo)

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,g1vsL_correlation,g2vsR_correlation
0,0,PELI1_T,SLC15A4_B,HMGB2,ITPR3,0.054634,0.398799,0.249186,0.111771,0.179812
1,1,SLA_T,AC025164.1_B,CXCL13,P2RY14,0.061751,0.394948,0.242504,0.445911,0.537213
2,2,IL6ST_T,CENPM_B,CXCL13,P2RY14,0.085231,0.368789,0.224098,0.329974,0.157364
3,3,IL7R_T,CEMIP2_B,FN1,SDC2,0.132547,0.357057,0.196633,-0.076033,0.716407
4,4,IL7R_T,CEMIP2_B,HMGB2,DSG2,0.148023,0.357057,0.189290,0.394970,0.727592
5,5,CEMIP2_T,SOCS1_B,FN1,SDC2,0.129849,0.353070,0.197978,0.004002,0.575700
6,6,CEMIP2_T,SOCS1_B,HMGB2,DSG2,0.143793,0.353070,0.191238,-0.008852,0.589796
7,7,PELI1_T,TNFSF8_B,HMGB2,ITPR3,0.057595,0.347179,0.246326,0.111771,0.100454
8,8,TXNIP_T,SESN1_B,FN1,SDC2,0.059959,0.344692,0.244126,0.026227,0.744928
9,9,TXNIP_T,SESN1_B,HMGB2,DSG2,0.072889,0.344692,0.233188,0.237128,0.758026


In [58]:
# remove the rows with g1vsL_correlation < 0.3 
results = results[(results['g1vsL_correlation'] > 0.2) & (results['g1vsL_correlation'] < 0.99)]
# remove the rows with g2vsR_correlation < 0.3 
results = results[(results['g2vsR_correlation'] > 0.2) & (results['g2vsR_correlation'] < 0.99)]
results

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,g1vsL_correlation,g2vsR_correlation
1,1,SLA_T,AC025164.1_B,CXCL13,P2RY14,0.061751,0.394948,0.242504,0.445911,0.537213
4,4,IL7R_T,CEMIP2_B,HMGB2,DSG2,0.148023,0.357057,0.189290,0.394970,0.727592
9,9,TXNIP_T,SESN1_B,HMGB2,DSG2,0.072889,0.344692,0.233188,0.237128,0.758026
10,10,TSC22D3_T,AC025164.1_B,CXCL13,P2RY14,0.054046,0.343079,0.249769,0.263070,0.537213
12,12,PELI1_T,AC025164.1_B,CXCL13,CCR7,0.056453,0.333916,0.247415,0.355799,0.402467
13,13,PELI1_T,AC025164.1_B,CXCL13,P2RY14,0.098882,0.333916,0.215171,0.355799,0.537213
15,15,PELI1_T,AC025164.1_B,CXCL13,GABBR1,0.067946,0.333916,0.237172,0.355799,0.283082
26,26,RGS1_T,KLF9_B,CCL5,CDH1,0.079513,0.306265,0.228171,0.341010,0.253403
31,31,NFKBIA_T,AC025164.1_B,CXCL13,CCR7,0.065009,0.300566,0.239649,0.212206,0.402467
32,32,NFKBIA_T,AC025164.1_B,CXCL13,P2RY14,0.123783,0.300566,0.201079,0.212206,0.537213


In [47]:
results['g1'] = results['g1'].str.replace(r'_\w+', '', regex=True)
results['g2'] = results['g2'].str.replace(r'_\w+', '', regex=True)
results

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,g1vsL_correlation,g2vsR_correlation
1,1,SLA,AC025164.1,CXCL13,P2RY14,0.061751,0.394948,0.242504,0.445911,0.537213
4,4,IL7R,CEMIP2,HMGB2,DSG2,0.148023,0.357057,0.189290,0.394970,0.727592
12,12,PELI1,AC025164.1,CXCL13,CCR7,0.056453,0.333916,0.247415,0.355799,0.402467
13,13,PELI1,AC025164.1,CXCL13,P2RY14,0.098882,0.333916,0.215171,0.355799,0.537213


In [48]:
# get the unique (L,R) pairs in results
lr_pairs = results[['L', 'R']]
# remove the LR pairs repeated less than 3 times in the lr_pairs
lr_pairs = lr_pairs.groupby(['L', 'R']).filter(lambda x: len(x) >= 1)
lr_pairs = lr_pairs.drop_duplicates()
lr_pairs

,L,R
1,CXCL13,P2RY14
4,HMGB2,DSG2
12,CXCL13,CCR7


In [49]:
import pandas as pd
grouped_results = results.groupby(["L", "R"])

# Display grouped data
grouped_dict = { (ligand, receptor): group for (ligand, receptor), group in grouped_results }

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
grouped_data = []
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 1

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1 and GEM_g2
    # and where the overlap is below the maximum allowable threshold
    if len(gem1) > min_gene_num and len(gem2) > min_gene_num:
        temp_data.append({
            "L": ligand,
            "R": receptor,
            "g1": gem1,
            "g2": gem2,
            "Num_genes_g1": len(gem1),
            "Num_genes_g2": len(gem2)
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)

# Function to find overlapping rows
def find_overlapping_rows(df, column_name):
    overlap_info = []
    for index, row in df.iterrows():
        current_set = set(row[column_name])
        overlapping_rows = []
        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_set = set(other_row[column_name])
                overlap = len(current_set & other_set)
                if overlap > len(current_set) / 3: # Set the threshold for overlap
                    overlapping_rows.append(other_index)
        overlap_info.append(overlapping_rows)
    return overlap_info

# Add columns for overlapping rows in GEM_g1 and GEM_g2
grouped_df["Overlapping_g1"] = find_overlapping_rows(grouped_df, "g1")
grouped_df["Overlapping_g2"] = find_overlapping_rows(grouped_df, "g2")


In [51]:
# rank each GEM_g1 by order of name inside
grouped_df['g1'] = grouped_df['g1'].apply(lambda x: sorted(x))
grouped_df['g2'] = grouped_df['g2'].apply(lambda x: sorted(x))
grouped_df

KeyError: 'g1'

In [20]:
# remove '-0' and '-1' in adata_1_pseudo.obs.index
adata_1_pseudo.obs.index = [i.split('-')[0] for i in adata_1_pseudo.obs.index]

In [21]:
#calculate the correlation between g1 and g2
calculate_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)
#calculate the residuals correlation between g1, g2 and LR
calculate_residuals_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,HMGB2,DSG2,"[IL7R, TXNIP]","[CEMIP2, SESN1]",2,2,[],[],0.513643,0.467611


In [22]:
grouped_df['GEM_1'] = 'GEM_' + grouped_df.index.astype(str) + '_CD4EX'
grouped_df['GEM_2'] = 'GEM_' + grouped_df.index.astype(str) + '_B'

In [23]:
# Define a function to append '_T' to each gene name
def append_suffix(gene_list, suffix='_T'):
    # Ensure it's a list and not NaN or another unexpected type
    if isinstance(gene_list, list):
        return [gene + suffix for gene in gene_list]
    else:
        print("Unexpected format:", gene_list)
        return gene_list

# Apply the function to the GEM_g1 column to add '_T' to each gene name
DEG_pairs_T = grouped_df.copy()
DEG_pairs_T['g1'] = grouped_df['g1'].apply(lambda x: append_suffix(x, '_CD4EX'))

# Similarly for GEM_g2 if needed
DEG_pairs_T['g2'] = grouped_df['g2'].apply(lambda x: append_suffix(x, '_B'))
# remove the genes not in gene_df from each row in DEG_pairs_T but keep the row
DEG_pairs_T['g1'] = DEG_pairs_T['g1'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])
DEG_pairs_T['g2'] = DEG_pairs_T['g2'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])


# add GEM_1_T, GEM_2_T and so on to gene_df
for i, row in DEG_pairs_T.iterrows():
    GEM_1 = DEG_pairs_T.loc[i, 'g1']
    GEM_2 = DEG_pairs_T.loc[i, 'g2']
    gene_df['GEM_' + str(i) + '_CD4EX'] = gene_df[GEM_1].mean(axis=1)
    gene_df['GEM_' + str(i) + '_B'] = gene_df[GEM_2].mean(axis=1)
gene_df

,ABCC10_CD4EX,AC004687.1_CD4EX,AC005224.3_CD4EX,AC016596.1_CD4EX,AC133644.2_CD4EX,ADAR_CD4EX,ADGRE5_CD4EX,AK1_CD4EX,ANKRD28_CD4EX,ANKRD55_CD4EX,...,IGKV1-13_B,AC103831.1_B,AL031733.2_B,AL662796.1_B,GALNTL6_B,LINC01685_B,treatment,patient_id,GEM_0_CD4EX,GEM_0_B
sample_id,,,,,,,,,,,,,,,,,,,,,
BIOKEY_10_On,180.927872,222.160614,88.135345,15.073813,239.997925,1511.042847,1342.128174,181.157166,1078.547607,89.435349,...,102.727951,63.48246,34.220703,59.604637,10.585213,119.495178,1,BIOKEY_10,2261.504639,847.418701
BIOKEY_10_Pre,145.154297,506.464264,17.049347,7.727849,60.681461,1440.893799,1794.357178,91.951340,284.963806,7.045348,...,210.157516,0.00000,0.000000,0.000000,0.000000,0.000000,0,BIOKEY_10,1186.045013,353.005859
BIOKEY_11_On,95.089760,167.290665,78.751785,39.231506,135.439026,1904.050293,878.242188,10.115088,539.381592,102.666580,...,97.603500,0.00000,108.897171,203.385483,0.000000,0.000000,1,BIOKEY_11,3268.564026,748.035522
BIOKEY_11_Pre,277.542297,536.481689,0.000000,0.000000,124.803528,1868.696533,1127.876465,0.000000,190.240128,0.000000,...,344.332397,0.00000,0.000000,0.000000,0.000000,0.000000,0,BIOKEY_11,1064.317078,336.206665
BIOKEY_12_On,178.689575,336.540131,142.999542,60.020634,89.445770,1888.558228,1078.586914,102.949722,758.446289,472.756531,...,0.000000,0.00000,177.698975,247.096985,279.785614,273.152374,1,BIOKEY_12,3226.974854,1549.362061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIOKEY_7_Pre,1417.506592,900.541626,0.000000,0.000000,0.000000,2318.048340,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0,BIOKEY_7,678.692322,436.871002
BIOKEY_8_On,191.187454,109.494469,37.834351,27.820389,77.332085,2182.324707,1512.138672,124.553421,350.137390,198.561462,...,0.000000,0.00000,0.000000,361.058746,0.000000,0.000000,1,BIOKEY_8,2238.919006,1035.200928
BIOKEY_8_Pre,956.909302,0.000000,0.000000,0.000000,0.000000,3833.964844,1147.319702,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0,BIOKEY_8,800.544708,0.000000


In [24]:
# add GEM_1_T, GEM_2_T and so on to adata
DEG_pairs = grouped_df
for i, row in DEG_pairs.iterrows():
    GEM_1 = DEG_pairs.loc[i, 'g1']
    GEM_2 = DEG_pairs.loc[i, 'g2']
    # add GEMs to adata_1_pseudo and adata_2_pseudo
    adata_1_pseudo.obs['GEM_' + str(i) + '_CD4EX'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
    adata_2_pseudo.obs['GEM_' + str(i) + '_B'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
    # add GEMs to adata_1 and adata_2
    adata_1.obs['GEM_' + str(i) + '_CD4EX'] = adata_1[:, GEM_1].X.mean(axis=1)
    adata_2.obs['GEM_' + str(i) + '_B'] = adata_2[:, GEM_2].X.mean(axis=1)

/tmp/ipykernel_1658871/205628274.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1.obs['GEM_' + str(i) + '_CD4EX'] = adata_1[:, GEM_1].X.mean(axis=1)


In [25]:
CIT_GEM_test(grouped_df, adata_1_pseudo, adata_2_pseudo, gene_df, method="fisherz")

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,GEM_1,GEM_2,pValue
0,HMGB2,DSG2,"[IL7R, TXNIP]","[CEMIP2, SESN1]",2,2,[],[],0.513643,0.467611,GEM_0_CD4EX,GEM_0_B,0.000129


In [26]:
def calculate_g1_L_GEM_correlation(results, adata_1_pseudo, method='pearson'):
    if method not in ['pearson', 'spearman']:
        raise ValueError("Method must be either 'pearson' or 'spearman'")
    
    results['GEM1vsL_correlation'] = 0

    for index, row in results.iterrows():
        g1 = row['GEM_1']
        L = row['L']


        g1_data = adata_1_pseudo.obs[g1]
        g1_df = pd.DataFrame(g1_data, columns=[g1], index=adata_1_pseudo.obs['sample_id'])
        g1_df = g1_df.dropna()

        L_data = adata_1_pseudo[:, L].X
        L_df = pd.DataFrame(L_data, columns=[L], index=adata_1_pseudo.obs['sample_id'])
        L_df = L_df.dropna()

        if method == 'pearson':
            corr_g1_g2 = pearsonr(g1_df[g1], L_df[L])[0]
        elif method == 'spearman':
            corr_g1_g2 = spearmanr(g1_df[g1], L_df[L])[0]

        results.loc[index, 'GEM1vsL_correlation'] = corr_g1_g2
    
    return results

In [27]:
def calculate_g2_R_GEM_correlation(results, adata_2_pseudo, method='pearson'):
    if method not in ['pearson', 'spearman']:
        raise ValueError("Method must be either 'pearson' or 'spearman'")
    
    results['GEM2vsR_correlation'] = 0

    for index, row in results.iterrows():
        g2 = row['GEM_2']
        R = row['R']

        g2_data = adata_2_pseudo.obs[g2]
        g2_df = pd.DataFrame(g2_data, columns=[g2], index=adata_2_pseudo.obs['sample_id'])
        g2_df = g2_df.dropna()

        R_data = adata_2_pseudo[:, R].X
        R_df = pd.DataFrame(R_data, columns=[R], index=adata_2_pseudo.obs['sample_id'])
        R_df = R_df.dropna()

        if method == 'pearson':
            corr = pearsonr(g2_df[g2], R_df[R])[0]
        elif method == 'spearman':
            corr = spearmanr(g2_df[g2], R_df[R])[0]

        results.loc[index, 'GEM2vsR_correlation'] = corr
    
    return results


In [28]:
def calculate_geneLR_GEM_correlation(results, adata_1_pseudo, adata_2_pseudo, method='pearson'):
    if method not in ['pearson', 'spearman']:
        raise ValueError("Method must be either 'pearson' or 'spearman'")
    
    results['GEM1vsLR_correlation'] = 0
    results['GEM2vsLR_correlation'] = 0

    for index, row in results.iterrows():
        g1 = row['GEM_1']
        g2 = row['GEM_2']
        L = row['L']
        R = row['R']


        g1_data = adata_1_pseudo.obs[g1]
        g1_df = pd.DataFrame(g1_data, columns=[g1], index=adata_1_pseudo.obs['sample_id'])
        g1_df = g1_df.dropna()

        g2_data = adata_2_pseudo.obs[g2]
        g2_df = pd.DataFrame(g2_data, columns=[g2], index=adata_2_pseudo.obs['sample_id'])
        g2_df = g2_df.dropna()

        L_data = adata_1_pseudo[:, L].X
        L_df = pd.DataFrame(L_data, columns=[L], index=adata_1_pseudo.obs['sample_id'])
        L_df = L_df.dropna()

        R_data = adata_2_pseudo[:, R].X
        R_df = pd.DataFrame(R_data, columns=[R], index=adata_2_pseudo.obs['sample_id'])
        R_df = R_df.dropna()

        L_df.columns = [R + '_vs_' + L]
        R_df.columns = L_df.columns

        L_R = L_df.multiply(R_df, axis=0)
        L_R = L_R.iloc[:, 0]

        if method == 'pearson':
            corr_g1_LR = pearsonr(g1_df[g1], L_R)[0]
            corr_g2_LR = pearsonr(g2_df[g2], L_R)[0]
        elif method == 'spearman':
            corr_g1_LR = spearmanr(g1_df[g1], L_R)[0]
            corr_g2_LR = spearmanr(g2_df[g2], L_R)[0]

        results.loc[index, 'GEM1vsLR_correlation'] = corr_g1_LR
        results.loc[index, 'GEM2vsLR_correlation'] = corr_g2_LR

    return results

In [29]:
calculate_g1_L_GEM_correlation(grouped_df, adata_1_pseudo)
calculate_g2_R_GEM_correlation(grouped_df, adata_2_pseudo)
calculate_geneLR_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,GEM_1,GEM_2,pValue,GEM1vsL_correlation,GEM2vsR_correlation,GEM1vsLR_correlation,GEM2vsLR_correlation
0,HMGB2,DSG2,"[IL7R, TXNIP]","[CEMIP2, SESN1]",2,2,[],[],0.513643,0.467611,GEM_0_CD4EX,GEM_0_B,0.000129,0.336069,0.823065,0.319412,0.825888


In [30]:
# Reorder columns
grouped_df = grouped_df[['L', 'R', 'GEM_1', 'GEM_2', 'g1', 'g2', 'Num_genes_g1', 'Num_genes_g2', 
                         'g1vsg2_correlation', 'g1_residualvsg2_residuals_correlation', 
                         'GEM1vsL_correlation', 'GEM2vsR_correlation', 'GEM1vsLR_correlation', 
                         'GEM2vsLR_correlation', 'pValue']]

# Rename column
grouped_df = grouped_df.rename(columns={'GEM_1': 'GEM_1 (X)'})
grouped_df = grouped_df.rename(columns={'GEM_2': 'GEM_2 (Y)'})
grouped_df = grouped_df.rename(columns={'g1vsg2_correlation': 'Corr (X-Y)'})
grouped_df = grouped_df.rename(columns={'g1_residualvsg2_residuals_correlation': 'Corr (cond.)'})
grouped_df = grouped_df.rename(columns={'GEM1vsL_correlation': 'Corr (X-L)'})
grouped_df = grouped_df.rename(columns={'GEM2vsR_correlation': 'Corr (R-Y)'})
grouped_df = grouped_df.rename(columns={'GEM1vsLR_correlation': 'Corr (X-LR)'})
grouped_df = grouped_df.rename(columns={'GEM2vsLR_correlation': 'Corr (LR-Y)'})
grouped_df = grouped_df.rename(columns={'pValue': 'p-value (CIT fisherz)'})


In [31]:
# save as csv
grouped_df.to_csv('/home/qiuaodon/Desktop/project_data_new/CIT_results_CD4EXvsB_grouped_0921.csv')